# Исследование надежности заемщиков


**Описание проекта**

Заказчик — кредитный отдел банка. Необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## Общая информация о данных

In [ ]:
# импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# чтение файла
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [ ]:
data.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5088,0,-547.889771,27,высшее,0,гражданский брак,1,F,сотрудник,0,142601.204718,сыграть свадьбу
12469,0,-1822.121087,29,среднее,1,женат / замужем,0,F,сотрудник,0,102228.874325,строительство недвижимости
10303,0,-1340.340467,54,среднее,1,гражданский брак,1,F,сотрудник,0,76343.513892,свадьба
13309,0,-1101.607552,50,высшее,0,гражданский брак,1,F,сотрудник,0,85413.882304,на проведение свадьбы
17054,0,-1835.865868,34,Среднее,1,Не женат / не замужем,4,M,сотрудник,0,202730.829811,недвижимость
20285,1,-339.503025,28,среднее,1,женат / замужем,0,F,сотрудник,0,80086.432507,операции с недвижимостью
21394,0,-2095.025283,26,высшее,0,Не женат / не замужем,4,M,сотрудник,1,211722.108212,автомобиль
9612,0,-1765.748588,54,среднее,1,женат / замужем,0,F,сотрудник,0,160810.509038,заняться высшим образованием
7825,1,-9280.884493,49,среднее,1,женат / замужем,0,F,компаньон,0,187454.884869,покупка своего жилья
2937,0,329555.513459,59,высшее,0,женат / замужем,0,M,пенсионер,0,93920.085956,заняться высшим образованием


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Значения в стобцах имеют разные типы данных, а также присутствуют пропуски 

## Предобработка данных

### Удаление пропусков в столбце income_type
Путем замены на медианное значение в соответствии с типом занятости

In [ ]:
# количество пропусков в стобцах
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
# заполнение пропусков в столбце total_income медианным значением в соответствии с типом занятости (income_type)
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений в столбцах days_employed и children

In [ ]:
# замена отрицательных значений в столбце days_employed на модуль от данного числа - устранение аномалий
data['days_employed'] = data['days_employed'].abs()

In [ ]:
# подсчет медианного значения стажа работы по типу занятости (в днях)
data.groupby('income_type')['days_employed'].agg('median').sort_values(ascending=False)

income_type
безработный        366413.652744
пенсионер          365213.306266
в декрете            3296.759962
госслужащий          2689.368353
сотрудник            1574.202821
компаньон            1547.382223
студент               578.751554
предприниматель       520.848083
Name: days_employed, dtype: float64

In [ ]:
# количество значений стажа (days_employed) для типа занятости (income_type) равного безработный и пенсионер 
data.loc[(data['income_type']=='пенсионер')|(data['income_type']=='безработный')].groupby('income_type')['days_employed'].count().sort_values(ascending=False).to_frame().reset_index()


,income_type,days_employed
0,пенсионер,3443
1,безработный,2


Можно обратить внимание на присутствие аномально высоких значений стажа -> около 1 000 лет для безработных и пенсионеров

При расчете максимального значения неприрывного стажа (от 18 лет до возраста выхода на пенсию) для женщин он составляет 37 лет - 13 505 дней, для мужчин - 42 года - это 15 330 дня

Исправить данные аномалии без дополнительных значений сложно, поэтому придеся оставить их в таком виде, кроме того люди, данного типа занятости, не могут взять кредит и таким образом нам не нужно рассчитывать их надежность.

In [ ]:
# выявление аномальных значений в столбце с количеством детей (children) 
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце с количеством детей (children) присутствует 2 аномальных значения - 20 детей и минус 1

In [ ]:
# удаляем аномальные значения из данных
data = data[(data['children'] != -1) & (data['children'] != 20)]

### Удаление пропусков в столбце days_employed
Путем замены медианным значением в соответствии с типом занятости


In [ ]:
# заполнение пропусков в столбце days_employed медианным значением в соответствии с типом занятости (income_type)
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Таким образом, были устранены все пропуски в данных, а также исправлены явные аномалии

### Изменение типов данных

In [ ]:
# замена вещественного типа данных в столбце `total_income` на целочисленный с помощью метода `astype()`
data['total_income'] = data['total_income'].astype(int)

<ipython-input-14-76d1683dce75>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income'] = data['total_income'].astype(int)


### Обработка дубликатов

In [ ]:
# количество явных дубликатов в данных
print('Количество дубликатов в данных -', data.duplicated().sum())

Количество дубликатов в данных - 54


In [ ]:
# удаление явных дубликатов
data = data.drop_duplicates()

In [ ]:
# анализ неявных дубликатов в столбце education
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [ ]:
# приведение значений в столбце education к нижнему регистру для устранения неявных дубликатов
data['education'] = data['education'].str.lower()
print('Уникальные значения в столбце `education` -', data['education'].unique())

Уникальные значения в столбце `education` - ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


<ipython-input-18-c3d5d6d9e33c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['education'] = data['education'].str.lower()


### Категоризация данных

Создание функции для разделения кредитополучателей на категории по доходу:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
# применение функции
data['total_income_category'] = data['total_income'].apply(categorize_income)

<ipython-input-20-068246cc922c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_income_category'] = data['total_income'].apply(categorize_income)


Разделение данных о целях взятия кредита (purpose) на более широкие понятия:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`

In [ ]:
# уникальные значения в стобце purpose
sorted(data['purpose'].unique())

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

In [ ]:
# создание функции для категоризации
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
# применение функции
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных


### Определение зависимости между количеством детей и возвратом кредита в срок

In [ ]:
# создание таблицы для анализа
# группировка df по количеству детей и выявление среди них заёмщиков, имеющих задолженность по возврату кредитов, а также
# определение общего количества заёмщиков

data_debtor=data[['children', 'debt', 'total_income']]
data_debtor=data_debtor.rename(columns = {'children':'number_of_children', 'debt':'clients_with_debt', 'total_income':'debtor_count'})

data_debtor=data_debtor.groupby('number_of_children',as_index=False).agg({'clients_with_debt':'sum', 'debtor_count':'count'})

In [ ]:
# определение процента не возвращающих кредит в срок от общего количества заёмщиков по опреленной группе
data_debtor['percent']=data_debtor['clients_with_debt']*100/data_debtor['debtor_count']
data_debtor=data_debtor.round({'percent':2}).sort_values(by='percent', ascending=False)
print('Распределение возврата кредита в зависимости от разного количества детей')
display(data_debtor)


Распределение возврата кредита в зависимости от разного количества детей


,number_of_children,clients_with_debt,debtor_count,percent
4,4,4,41,9.76
2,2,194,2052,9.45
1,1,444,4809,9.23
3,3,27,330,8.18
0,0,1063,14107,7.54
5,5,0,9,0.00


In [ ]:
# создание df, который делит общее число имеющих задолженность по возврату кредита на 2 группы - с детьми и без детей
data_kids=data_debtor.groupby(data_debtor['number_of_children']>0, as_index=False)[['clients_with_debt', 'debtor_count']].agg('sum')
data_kids['percent']=data_kids['clients_with_debt']*100/data_kids['debtor_count']
data_kids=data_kids.round({'percent':2}).sort_values(by='percent', ascending=False)
data_kids.index = ['Have kids', "Don't have kids"]
print('Распределение возврата кредита в зависимости от наличия детей')
display(data_kids)


Распределение возврата кредита в зависимости от наличия детей


,clients_with_debt,debtor_count,percent
Have kids,669,7241,9.24
Don't have kids,1063,14107,7.54


In [ ]:
print('Статистика')
print()
total_debt=data_kids.clients_with_debt.sum()
print(total_debt, "чел. - общее число заёмщиков, имеющих задолженность по возврату кредита")
total_people=data_kids.debtor_count.sum()
print(total_people, "чел. - общее число заёмщиков")
persent_of_debt=total_debt*100/total_people
print(round(persent_of_debt,2),'% от общего числа заёмщиков имели задолженность по возврату кредита')
print()
print(int(data_kids.loc[data_kids.index=="Don't have kids"]['clients_with_debt']), 'чел., имеющих задолженность по возврату кредита были без детей')
print(int(data_kids.loc[data_kids.index=="Have kids"]['clients_with_debt']), 'чел., имеющих задолженность по возврату кредита были с детьми')
print()
persent_of_debt_no_kids=(int(data_kids.loc[data_kids.index=="Don't have kids"]['clients_with_debt']))*100/total_people
persent_of_debt_have_kids=(int(data_kids.loc[data_kids.index=='Have kids']['clients_with_debt']))*100/total_people
print(round(persent_of_debt_no_kids,2),'% от общего числа заёмщиков имели задолженность по возврату кредита и были без детей')
print(round(persent_of_debt_have_kids,2),'% от общего числа заёмщиков имели задолженность по возврату кредита и были с детьми')

Статистика

1732 чел. - общее число заёмщиков, имеющих задолженность по возврату кредита
21348 чел. - общее число заёмщиков
8.11 % от общего числа заёмщиков имели задолженность по возврату кредита

1063 чел., имеющих задолженность по возврату кредита были без детей
669 чел., имеющих задолженность по возврату кредита были с детьми

4.98 % от общего числа заёмщиков имели задолженность по возврату кредита и были без детей
3.13 % от общего числа заёмщиков имели задолженность по возврату кредита и были с детьми


**Вывод**

От общего числа должников (21 348 чел.) лишь 8% (1 732 чел.) имели задолженность по возврату кредита, из них 3.1% (669 чел.) были с детьми, а 4.9% (1 063 чел.) без детей.
   
   Разница между наличием и отсутствием детей в выбранной целевой группе(debt=1), составляет 1.8 п.п. (394 человека). Это позволяет сказать, что большее число имевших задолженность по возврату кредита были без детей. 
   Однако если смотреть на процент в целевой группе, можно заметить, что из общего числа заемщиков без детей, лишь 7.54% человек имели проблемы с возвратом кредита, а из числа заёмщиков с детьми - 9.24%. Также в первой таблице видно, что с увеличением количества детей растет количество заёмщиков с задолженностью по возврату кредита, однако наблюдается снижение данного показателя у заёмщиков с 3 и 5 детьми.
   Таким образом, наличие детей отрицательно влияет на возврат кредита. (не большая разница в %% - не явный вывод по выборке)

### Определение зависимости между семейным положением и возвратом кредита в срок

In [ ]:
# создание таблицы для анализа - группировка df по семейному статусу, определение для каждой категории количества заёмщиков, 
# имеющих задолженность по возврату кредита, а также определение общего количества заёмщиков, имеющих определенный семейный статус
data_debtor_family=data[['family_status', 'debt', 'total_income']]
data_debtor_family=data_debtor_family.groupby('family_status',as_index=False)\
                                     .agg({'debt':'sum', 'total_income':'count'})
data_debtor_family=data_debtor_family.rename(columns = {'debt':'clients_with_debt', 'total_income':'debtor_count'})

In [ ]:
# определение процента не возвращающих кредит в срок от общего количества заёмщиков по опреленной группе
data_debtor_family['percent']=data_debtor_family['clients_with_debt']*100/data_debtor_family['debtor_count']
data_debtor_family=data_debtor_family.round({'percent':2})
data_debtor_family=data_debtor_family.sort_values(by='percent', ascending=False)
data_debtor_family.head()

,family_status,clients_with_debt,debtor_count,percent
0,Не женат / не замужем,273,2796,9.76
3,гражданский брак,385,4146,9.29
4,женат / замужем,927,12266,7.56
1,в разводе,84,1189,7.06
2,вдовец / вдова,63,951,6.62


In [ ]:
# создание df, который делит общее число имеющих задолженность по возврату кредитов на 2 группы:
#  - Married: в браке
#  - Not married: не в браке 

data_family=data_debtor_family.groupby((data_debtor_family['family_status']!='гражданский брак') & \
                                       (data_debtor_family['family_status']!= 'женат / замужем'),\
                                       as_index=True)[['clients_with_debt', 'debtor_count']].agg('sum')
data_family['percent']=data_family['clients_with_debt']*100/data_family['debtor_count']
data_family=data_family.round({'percent':2}).sort_values(by='percent', ascending=False)
data_family.index = ['Not married', "Married"]
data_family


,clients_with_debt,debtor_count,percent
Not married,420,4936,8.51
Married,1312,16412,7.99


In [ ]:
print('Статистика')
print()
total_debt=data_family.clients_with_debt.sum()
print(total_debt, "чел. - общее число заёмщиков, имеющих задолженность по возврату кредита")
total_people=data_family.debtor_count.sum()
print(total_people, "чел. - общее число заёмщиков")
persent_of_debt=total_debt*100/total_people
print(round(persent_of_debt,2),'% от общего числа заёмщиков имели задолженность по возврату кредита')
print()
print(int(data_family.loc[data_family.index=='Married']['clients_with_debt']), 'чел., имеющих задолженность по возврату кредита, были в браке')
print(int(data_family.loc[data_family.index=='Not married']['clients_with_debt']), 'чел., имеющих задолженность по возврату кредита, не в браке')
print()
persent_of_debt_married=(int(data_family.loc[data_family.index=='Married']['clients_with_debt']))*100/total_people
persent_of_debt_not_married=(int(data_family.loc[data_family.index=='Not married']['clients_with_debt']))*100/total_people
print(round(persent_of_debt_married,2),'% от общего числа заёмщиков имели задолженность по возврату кредита и находятся в браке')
print(round(persent_of_debt_not_married,2),'% от общего числа заёмщиков имели задолженность по возврату кредита и не находятся в браке')

Статистика

1732 чел. - общее число заёмщиков, имеющих задолженность по возврату кредита
21348 чел. - общее число заёмщиков
8.11 % от общего числа заёмщиков имели задолженность по возврату кредита

1312 чел., имеющих задолженность по возврату кредита, были в браке
420 чел., имеющих задолженность по возврату кредита, не в браке

6.15 % от общего числа заёмщиков имели задолженность по возврату кредита и находятся в браке
1.97 % от общего числа заёмщиков имели задолженность по возврату кредита и не находятся в браке


**Вывод:**

От общего числа должников (21 348 чел.) лишь 8% (1732 чел.) имели задолженность по возврату кредита, из них 6.15% (1 312 чел.) находятся в браке, а 1.97% (420 чел.) нет.

Разница между наличием и отсутствием брака в выбранной целевой группе(debt=1), составляет 4.18 п.п. (892 человека). Это позволяет сказать, что большее число имевших задолженность по возврату кредита находятся в браке.
Однако если смотреть на процент в целевой группе, можно заметить, что из общего числа заемщиков в браке, лишь 7.99% человек имели проблемы с возвратом кредита, а из числа заёмщиков не в браке - 8.51%. Таким образом, отсутствие брака отрицательно влияет на возврат кредита. (но %% почти равны - не точный вывод по выборке)

### Определение зависимости между уровнем дохода и возвратом кредита в срок

In [ ]:
# создание таблицы для анализа
data_debtor_income=data[['total_income_category', 'debt', 'total_income']]
# группировка по категории и переименовывание колонок
data_debtor_income = data_debtor_income.groupby('total_income_category', as_index=True)\
                                       .agg({'debt':"sum",'total_income':"count"})
data_debtor_income=data_debtor_income.rename(columns = {'debt':'clients_with_debt', 'total_income':'debtor_count'})

In [ ]:
# определение процента не возвращающих кредит в срок от общего количества заёмщиков по опреленной группе
data_debtor_income['percent']=data_debtor_income['clients_with_debt']*100/data_debtor_income['debtor_count']
data_debtor_income=data_debtor_income.round({'percent':2})
data_debtor_income=data_debtor_income.sort_values(by='percent', ascending=False)
data_debtor_income

,clients_with_debt,debtor_count,percent
total_income_category,,,
E,2,22,9.09
C,1353,15938,8.49
A,2,25,8.00
B,354,5014,7.06
D,21,349,6.02


**Вывод:** 

0 – 30 000 — 'E' низкий уровень дохода

30 001 – 50 000 — 'D' низкий уровень дохода

50 001 – 200 000 — 'C' средний уровень дохода

200 001 – 1 000 000 — 'B' высокий уровень дохода
 
1 000 001 и выше — 'A' высокий уровень дохода

Выборка была поделена на группы по уровню дохода (подробнее выше). 

Наибольший процент (9.09%) от имевших задолженность по возврату кредита, у заёмщиков, обладающих низким уровнем дохода (0 – 30 000 руб.), однако количество людей этой группы очень не значительно, что не позволяет использовать данную группу для анализа.

По общему количеству видно, что большая часть заёмщиков - люди со средним уровнем дохода (50 001–200 000 руб.), 8.49% заёмщиков данной группы имели задолженность по возврату кредита, что является самым значительным результатом из всей выборки. 

### Определение зависимости между разными целями кредита и возвратом кредита в срок

In [ ]:
# создание таблицы для анализа
data_purpose=data[['purpose_category','debt','total_income']]
# группировка и переименовывание колонок
data_purpose=data_purpose.groupby('purpose_category')\
                         .agg({'debt':'sum','total_income':'count'})
data_purpose=data_purpose.rename(columns = {'debt':'clients_with_debt', 'total_income':'debtor_count'})


In [ ]:
# определение процента не возвращающих кредит в срок от общего количества заёмщиков по опреленной группе
data_purpose['percent']=data_purpose['clients_with_debt']*100/data_purpose['debtor_count']
data_purpose = data_purpose.round({'percent' : 2})
data_purpose=data_purpose.sort_values(by='percent', ascending=False)
data_purpose

,clients_with_debt,debtor_count,percent
purpose_category,,,
операции с автомобилем,400,4281,9.34
получение образования,369,3989,9.25
проведение свадьбы,183,2324,7.87
операции с недвижимостью,780,10754,7.25


**Вывод:** 
Целью для взятия кредита у наибольшего процента заёмщиков (9.34% и 9.25%), имевших задолженность по возврату кредита, было - операции с автомобилем	и получение образования.

## Возможные причины появления пропусков в исходных данных


1) Данные неизвестны

2) Сокрытие данных

3) Ошибка при заполнении

**Касательно данного кейса:**

Отобрав данные по пропускам по столбцу days_employed, было обнаружено, что значения симметричны пропускам по стобику total_income. Наиболее вероятно, что данные были не верно собраны или обработаны в этой части выборки, может быть были сбои в базе и данные не записались.

## Заполнение пропусков медианным значением для количественных переменных

В случае, если есть сомнения в нормальности распределения данных, предпочтительнее использовать медиану для заполнения пропусков. Так как выдающиеся значения в выборке могут исказить результат. Медиана более устойчива к выбросам.

## Общий вывод.

Изучив данные, можно сделать вывод о заёмщиках, имевших задолженность по возврату кредита. Чаще невозвращали в срок заёмщики: 

1) у которых есть дети (чаще 4 или 2 ребенка)

2) которые не женаты/замужем, либо были в гражданском браке(однако при разделении на 2 группы - в браке и не в браке - результат показал, что больший процент (8.51%) это заёмщики не находящиеся в браке)

3) доход таких заёмщиков средний (ниже 142 594 руб.)

4) целью взятия кредита являются - операции с автомобилем и получение образования

Наиболее ответственный заёмщик для банка - не имеет детей, вдова или вдовец, либо в разводе, имеет высокий доход, целью взятия кредита являются операции с недвижимостью. 

